In [ ]:
# Install necessary libraries
!pip install gspread oauth2client pandas matplotlib

In [ ]:
# Import required libraries
import requests
import pandas as pd
import time
from google.colab import auth
from oauth2client.service_account import ServiceAccountCredentials
import gspread

In [ ]:
# Step 1: Authentication for Google Sheets
auth.authenticate_user()

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving magnetic-guild-438706-j2-60cda877332b.json to magnetic-guild-438706-j2-60cda877332b.json


In [ ]:
# Provide a link to authenticate with Google Drive
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(
    'magnetic-guild-438706-j2-60cda877332b.json', scope)
client = gspread.authorize(creds)

In [ ]:
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('magnetic-guild-438706-j2-60cda877332b.json', scope)
client = gspread.authorize(creds)
print("Authenticated successfully!")

Authenticated successfully!


In [ ]:
# Create or open an existing Google Sheet
sheet_name = "Crypto_Live_Data"
try:
    sheet = client.open(sheet_name)  # Open the existing sheet
except gspread.exceptions.SpreadsheetNotFound:
    sheet = client.create(sheet_name)  # Create a new sheet if not found
    sheet.share('tnfxwitcher@gmail.com', perm_type='user', role='writer')

# Ensure a worksheet exists; if not, add a new one
try:
    worksheet = sheet.get_worksheet(0)  # Access the first worksheet
except IndexError:
    worksheet = sheet.add_worksheet(title="Sheet1", rows="100", cols="20")  # Add a new worksheet if none exist

In [ ]:
# Step 2: Fetch Live Cryptocurrency Data
def fetch_crypto_data():
    API_URL = "https://api.coingecko.com/api/v3/coins/markets"
    PARAMS = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 50,
        "page": 1,
        "sparkline": False,
    }
    response = requests.get(API_URL, params=PARAMS)
    return response.json()

In [ ]:
# Step 3: Analyze Data
def analyze_crypto_data(data):
    df = pd.DataFrame(data)
    df = df[["name", "symbol", "current_price", "market_cap", "total_volume", "price_change_percentage_24h"]]

    # Analysis
    top_5 = df.nlargest(5, "market_cap")[["name", "market_cap"]]
    avg_price = df["current_price"].mean()
    highest_change = df.nlargest(1, "price_change_percentage_24h")[["name", "price_change_percentage_24h"]]
    lowest_change = df.nsmallest(1, "price_change_percentage_24h")[["name", "price_change_percentage_24h"]]

    return df, top_5, avg_price, highest_change, lowest_change

In [ ]:
# Step 4: Update Google Sheet
def update_google_sheet(df, top_5, avg_price, highest_change, lowest_change):
    worksheet.clear()  # Clear existing data

    # Update live data
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())

    # Add analysis results below live data
    analysis_data = [
        ["", "", "", "Analysis Results"],
        ["Metric", "Value"],
        ["Top 5 Cryptocurrencies by Market Cap", top_5.to_string(index=False)],
        ["Average Price of Top 50 Cryptocurrencies", f"${avg_price:.2f}"],
        ["Highest 24-hour Price Change", highest_change.to_string(index=False)],
        ["Lowest 24-hour Price Change", lowest_change.to_string(index=False)],
    ]
    worksheet.append_rows(analysis_data)

In [ ]:
# Step 5: Main Loop to Continuously Fetch and Update
while True:
    try:
        print("Fetching live cryptocurrency data...")
        data = fetch_crypto_data()
        df, top_5, avg_price, highest_change, lowest_change = analyze_crypto_data(data)
        update_google_sheet(df, top_5, avg_price, highest_change, lowest_change)
        print("Google Sheet updated successfully.")

        time.sleep(300)  # Wait 5 minutes before the next update
    except Exception as e:
        print(f"An error occurred: {e}")
        break

Fetching live cryptocurrency data...
Google Sheet updated successfully.
Fetching live cryptocurrency data...
Google Sheet updated successfully.
Fetching live cryptocurrency data...
Google Sheet updated successfully.


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-c23cc25e0c5e>", line 10, in <cell line: 2>
    time.sleep(300)  # Wait 5 minutes before the next update
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.10/dist-pa

TypeError: object of type 'NoneType' has no len()